http://maljnutr.org.my/

In [47]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import json
import json
import urllib3
import re
from urllib.parse import quote
from urllib.parse import unquote
import os

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import fitz

In [38]:
full_pdf_urls = []
year_lists = ["online-first", 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002, 2001, 2000, 1999, 1998, 1997, 1996, 1995]
for year in year_lists:
    url = f"http://maljnutr.org.my/{year}.php"
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "lxml")
    a = soup.find_all('a', 'btn')
    base_link = "http://maljnutr.org.my/"
    a = [a_.get('href') for a_ in a]
    
    for link_suffix in a:
        pattern = r'publication/(\d+-\d+)/(.+)\.pdf'
        match = re.match(pattern, link_suffix)
        if match:
            volume_issue = match.group(1)
            article_info = match.group(2)
            encoded_string = quote(article_info)
            full_pdf_url = base_link + 'publication/' + volume_issue + '/' + encoded_string + '.pdf'
            full_pdf_urls.append(full_pdf_url)
        else:
            print("No match found.")

No match found.
No match found.
No match found.
No match found.
No match found.
No match found.
No match found.
No match found.
No match found.
No match found.
No match found.


In [39]:
full_pdf_urls

['http://maljnutr.org.my/publication/27-3/Vol%2027%283%29%201.mjn.2021.0011%20Chang.pdf',
 'http://maljnutr.org.my/publication/27-3/Vol%2027%283%29%203.mjn.2020.0120%20Zainal%20Arifen%20%28online%20first%29.pdf',
 'http://maljnutr.org.my/publication/27-3/Vol%2027%283%29%202.mjn.2020.0118%20Hamirudin%20%28online%20first%29.pdf',
 'http://maljnutr.org.my/publication/27-3/Vol%2027%283%29%204.mjn.2021.0010%20Domingo%20%28online%20first%29.pdf',
 'http://maljnutr.org.my/publication/27-3/Vol%2027%283%29%205.mjn.2021.0009%20Kuruvilla%20%28online%20first%29.pdf',
 'http://maljnutr.org.my/publication/27-3/Vol%2027%283%29%206.mjn.2021.0045%20Rahman%20%28online%20first%29.pdf',
 'http://maljnutr.org.my/publication/27-1/MJN%20Vol%2027%20No%201_April%202021%20%28Final%20Full%20Compilation%29.pdf',
 'http://maljnutr.org.my/publication/27-1/Vol%2027%281%29%201.%20mjn.2020.0063%20Rakib%20Hasan%20%28final%29.pdf',
 'http://maljnutr.org.my/publication/27-1/Vol%2027%281%29%202.%20mjn.2020.0070%20Syahrul%

In [58]:
with open('full_pdf_urls.json', 'w') as fopen:
    json.dump(full_pdf_urls, fopen)

In [59]:
len(full_pdf_urls)

986

In [60]:
def get_filename_from_url(url):
    pattern = r'/publication/(\d+-\d+)/(.+)\.pdf'
    match = re.search(pattern, url)

    if match:
        volume_issue = match.group(1)
        filename_without_extension = match.group(2)
        return filename_without_extension
    else:
        print("No match found.")

In [61]:
for link in tqdm(full_pdf_urls):
        r = requests.get(link)
        filename = get_filename_from_url(link)
        filename = unquote(filename)
        with open(f'{filename}.pdf', 'wb') as pdf_file:
                pdf_file.write(r.content)

100%|██████████| 986/986 [16:36<00:00,  1.01s/it]  


In [64]:
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as pdf_document:
        num_pages = pdf_document.page_count

        for page_num in range(num_pages):
            page = pdf_document[page_num]
            text += page.get_text()

    return text


current_directory = os.getcwd()

data = []
for filename in os.listdir(current_directory):
    if filename.endswith(".pdf"):
        try:
            extracted_text = extract_text_from_pdf(filename)
            data.append({'filename': filename, 'texts': extracted_text})
        except:
            print("Error with this file: ", filename)

Error with this file:  a.pdf
Error with this file:  acknowledgements.pdf
Error with this file:  b.pdf
Error with this file:  c.pdf
Error with this file:  content.pdf
Error with this file:  d.pdf
Error with this file:  e.pdf
Error with this file:  f.pdf
Error with this file:  g.pdf
Error with this file:  h.pdf
Error with this file:  i.pdf
Error with this file:  j.pdf
Error with this file:  k.pdf
Error with this file:  l.pdf
Error with this file:  m.pdf
Error with this file:  q.pdf
Error with this file:  Vol 27(1) 7.mjn.2020.0074 Sandra (final).pdf


In [65]:
with open('maljnutr-pdf-extracted.json', 'w') as outfile:
    json.dump(data, outfile)